#### Abreviaturas:
- df: data frame
- cg_coef: coeficientes de Clebsch Gordan para SU(3)
- dtot: número de degeneraciones totales

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Se importa el archivo
df = pd.read_csv('8x8comp.txt', skiprows=4, header=None)

# Se agregan las cabeceras de las columnas
df.columns = ['klm', 'k1l1m1', 'k2l2m2', 'cg_coef', 'multiplet', 'degeneracy', 'dtot']

# Se eliminan las filas con valores NaN
df = df.dropna()

# Convierte las columnas "degeneracy" y "dtot" en tipo int
df = df.astype({"degeneracy":int})
df = df.astype({"dtot":int})

# Retorna una lista con los multipletes existentes
multiplets = df['multiplet'].value_counts().index.tolist() 

In [3]:
# Función para separar el df en multipletes
def divide_df(multiplet_i, dataf, headerc):
    df_multiplet = dataf[dataf[headerc] == multiplet_i]
    return df_multiplet

In [4]:
from sympy import *

In [5]:
# Calcula el cgc para un caso específico dentro de un dfe previamente filtrado
def cgc_md(initial1, initial2, final, ddf):
    dff = ddf.query('k1l1m1 == @initial1').query('k2l2m2 == @initial2').query('klm == @final')
    cgc_str=dff.loc[:,'cg_coef'].values[0]
    cgc = parse_expr(cgc_str, evaluate=0)
    return cgc

# Verifica si el df filtrado es simétrico o antisimétrico
def is_symmetric(sdf):
    p1 = sdf['k1l1m1'].head(1).values[0]
    p2 = sdf['k2l2m2'].head(1).values[0]
    pf = sdf['klm'].head(1).values[0]
    val1 = cgc_md(p1, p2, pf, sdf)
    val2 = cgc_md(p2, p1, pf, sdf)
    if val1 == val2:
        return 'S'
    else:
        return 'A'

In [6]:
Symmetry = []
Multiplets = []
Degeneracy = []
Index = []
for i in range(len(multiplets)):
    # Retorna un df del multiplete indicado en la función
    m = divide_df(multiplets[i], df, 'multiplet')
    
    # Retorna una lista con las degeneraciones del multiplete m
    degeneracy = m['degeneracy'].value_counts().index.tolist()
    
    if len(degeneracy)>1:
        for j in range(len(degeneracy)):
            Multiplets.append(multiplets[i])
            
            # Divide el multiplete en df por degeneraciones
            d = divide_df(degeneracy[j], m, 'degeneracy')
            
            Index.append(d.index)
            Symmetry.append(is_symmetric(d))
            Degeneracy.append(degeneracy[j])
    else:
        Index.append(m.index)
        Multiplets.append(multiplets[i])
        Symmetry.append(is_symmetric(m))
        Degeneracy.append(degeneracy[0])

# Elabora un df para describir cada multiplete
df_describe = pd.DataFrame()
df_describe['multiplet'] = Multiplets
df_describe['degeneracy'] = Degeneracy
df_describe['is_symm'] = Symmetry
df_describe

,multiplet,degeneracy,is_symm
0,11,2,S
1,11,1,A
2,22,1,S
3,30,1,A
4,03,1,A
5,00,1,S


In [33]:
df_describe['multiplet'].values #array
multiplets #lista

['11', '22', '30', '03', '00']

In [8]:
# Agrega una columna extra al df original
extra =[]
for i in range(df.shape[0]):
    extra.append('x')
df.loc[:,'is_symm']=extra
df

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
0,000,100,212,sqrt(1/8),00,1,1,x
1,000,101,211,-sqrt(1/8),00,1,1,x
2,000,111,111,-sqrt(1/8),00,1,1,x
3,000,200,202,sqrt(1/8),00,1,1,x
4,000,201,201,-sqrt(1/8),00,1,1,x
...,...,...,...,...,...,...,...,...
303,301,211,201,-sqrt(1/3),30,1,1,x
304,301,212,200,-sqrt(1/6),30,1,1,x
305,302,211,202,-sqrt(1/6),30,1,1,x
306,302,212,201,-sqrt(1/3),30,1,1,x


In [9]:
# Asigna correctamente la característica si es simétrico o antisimétrico
for i in range(len(Index)):
    for j in Index[i]:
        df.at[Index[i],'is_symm']=df_describe.loc[i, 'is_symm']
df

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
0,000,100,212,sqrt(1/8),00,1,1,S
1,000,101,211,-sqrt(1/8),00,1,1,S
2,000,111,111,-sqrt(1/8),00,1,1,S
3,000,200,202,sqrt(1/8),00,1,1,S
4,000,201,201,-sqrt(1/8),00,1,1,S
...,...,...,...,...,...,...,...,...
303,301,211,201,-sqrt(1/3),30,1,1,A
304,301,212,200,-sqrt(1/6),30,1,1,A
305,302,211,202,-sqrt(1/6),30,1,1,A
306,302,212,201,-sqrt(1/3),30,1,1,A


### Algoritmo para el cálculo de coeficientes

In [20]:
# Transforma los números cuánticos de una particula de y, i, iz al tipo k, l, m
def transform(particle):
    # Separa una palabra por caracteres y luego convierte cada elemento a entero
    p = int(list('11')[0])
    q = int(list('11')[1])
    
    # Convierte a caracter lo que convirtio en entero
    k = str(int((p+2*q)/3 + particle[0]/2 + particle[1]))
    l = str(int((p+2*q)/3 + particle[0]/2 - particle[1]))
    m = str(int((p+2*q)/3 + particle[0]/2 + particle[2]))
    
    # Une los caracteres klm en una sola cadena
    klm = ''.join([k, l, m])
    return klm

# Transforma una lista de lista de numeros
def transform_m(listm):
    list_str = []
    for i in range(len(listm)):
        list_str.append(transform(listm[i]))
    return list_str

In [24]:
# Octete de bariones
neutron, proton = [1, 1/2, -1/2], [1, 1/2, 1/2]
Sigma_mas, Sigma0, Sigma_menos = [1, 0, -1], [1, 0, 0], [1, 0, 1]
Lambda = [0,0,0]
Xi_menos, Xi_mas = [-1, 1/2, -1/2], [-1, 1/2, 1/2]
Barion8 = [proton, neutron, Sigma_mas, Sigma0, Sigma_menos, Lambda, Xi_menos, Xi_mas]

# Octete de bariones primados
Barion8_prime = B8

# Octede de Dibariones
Dn, Dp = [1, 1/2, -1/2], [1, 1/2, 1/2]
DS_mas, DS0, DSmenos = [1, 0, -1], [1, 0, 0], [1, 0, 1]
DL = [0,0,0]
DXmenos, DXmas = [-1, 1/2, -1/2], [-1, 1/2, 1/2]

Dibarion8 = [Dn, Dp, DS_mas, DS0, DSmenos, DL, DXmenos, DXmas]

B8 = transform_m(Barion8)
B8prime = B8
B8

['212', '211', '110', '111', '112', '111', '100', '101']

In [25]:
D8 = transform_m(Dibarion8)
D8

['211', '212', '110', '111', '112', '111', '100', '101']

In [12]:
df.loc[(df['multiplet']=='11')&(df['is_symm']=='A')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
58,100,100,111,-sqrt(1/4),11,1,2,A
59,101,101,111,-sqrt(1/4),11,1,2,A
60,100,100,201,-sqrt(1/12),11,1,2,A
61,100,101,200,sqrt(1/6),11,1,2,A
62,101,100,202,-sqrt(1/6),11,1,2,A
63,101,101,201,sqrt(1/12),11,1,2,A
64,100,111,100,sqrt(1/4),11,1,2,A
65,101,111,101,sqrt(1/4),11,1,2,A
66,100,200,101,-sqrt(1/6),11,1,2,A
67,100,201,100,sqrt(1/12),11,1,2,A


In [13]:
df.loc[(df['multiplet']=='11')&(df['klm']=='212')&(df['is_symm']=='S')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
154,212,111,212,-sqrt(1/20),11,2,2,S
157,212,201,212,sqrt(3/20),11,2,2,S
158,212,202,211,-sqrt(3/10),11,2,2,S
160,212,212,111,-sqrt(1/20),11,2,2,S
163,212,211,202,-sqrt(3/10),11,2,2,S
164,212,212,201,sqrt(3/20),11,2,2,S


In [14]:
df.loc[(df['klm']=='211')&(df['k1l1m1']=='111')&(df['k2l2m2']=='100')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
176,211,111,100,sqrt(9/20),22,1,1,S


In [15]:
df.loc[(df['multiplet']=='11')&(df['klm']=='212')&(df['is_symm']=='S')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
154,212,111,212,-sqrt(1/20),11,2,2,S
157,212,201,212,sqrt(3/20),11,2,2,S
158,212,202,211,-sqrt(3/10),11,2,2,S
160,212,212,111,-sqrt(1/20),11,2,2,S
163,212,211,202,-sqrt(3/10),11,2,2,S
164,212,212,201,sqrt(3/20),11,2,2,S


In [27]:
df.loc[(df['klm']=='212')&(df['k1l1m1']=='111')&(df['k2l2m2']=='212')&(df['is_symm']=='S')]

,klm,k1l1m1,k2l2m2,cg_coef,multiplet,degeneracy,dtot,is_symm
154,212,111,212,-sqrt(1/20),11,2,2,S


## TO DO
1. Empezar a formar la matriz de coeficientes de decaimiento con el algoritmo
2. Dp_lambda_p = [cgc(klm, k1l1m1, k2l2m2, octete, S)]

Otra forma de buscar un valor dentro de una columna de un df es:
~~~
df[df['multiplet'].str.contains('00')]
~~~

In [31]:
x=Symbol('x')
d1 = [sqrt(2)*4, x**2/x, Rational(3,4), 'nombre_decaimiento']

In [35]:
Matrix([d1[0:3]])

NameError: name 'd1' is not defined

In [150]:
e = Matrix([3,1])

In [151]:
if e[0]>e[1]:
    print('p>q')
else:
    print('q>p')
Imax = (e[0]+e[1])/2
print('Imax ', Imax)
Y = (e[0]-e[1])/3
print('Ymax ', Y)
Range = 2*Imax +1
print('Range ', Range)

p>q
Imax  2
Ymax  2/3
Range  5


In [168]:
Particles =[]

def dec_su2(nmax):
    row = []
    size_m = 2*nmax + 1
    for i in range(size_m):
        row.insert(0,nmax) 
        nmax = nmax -1
    return row

max_line = dec_su2(Imax)
Particles.append(max_line)
Particles

[[-2, -1, 0, 1, 2]]

In [170]:
Imax = (e[0]+e[1])/2
prow = []
Inew = Imax
for i in range(e[0]):
    Inew = nsimplify(Inew -1/2)
    prow.append(Inew)
prow

[3/2, 1, 1/2]

In [171]:
for i in range(len(prow)):
    Particles.append(dec_su2(prow[i]))
Particles

[[-2, -1, 0, 1, 2], [-3/2, -1/2, 1/2, 3/2], [-1, 0, 1], [-1/2, 1/2]]

In [172]:
Imax = (e[0]+e[1])/2
qrow = []
Inew = Imax
for i in range(e[1]):
    Inew = nsimplify(Inew -1/2)
    qrow.insert(0,Inew)
qrow

[3/2]

In [173]:
for i in range(len(qrow)):
    Particles.insert(0,dec_su2(qrow[i]))
Particles

[[-3/2, -1/2, 1/2, 3/2],
 [-2, -1, 0, 1, 2],
 [-3/2, -1/2, 1/2, 3/2],
 [-1, 0, 1],
 [-1/2, 1/2]]

In [136]:
nsimplify(0.5)

1/2

In [108]:
i = Matrix([1,0])
k = Matrix([Rational(1,2), sqrt(3)/2])
l = Matrix([-Rational(1,2), sqrt(3)/2])

In [94]:
pi = e - 2*i*e.dot(i)
pi

Matrix([
[-2],
[ 1]])

In [95]:
pk = e - 2*k*e.dot(k)
pk

Matrix([
[               1 - sqrt(3)/2],
[-sqrt(3)*(sqrt(3)/2 + 1) + 1]])

In [96]:
pl = e - 2*l*e.dot(l)
pl

Matrix([
[                sqrt(3)/2 + 1],
[-sqrt(3)*(-1 + sqrt(3)/2) + 1]])

In [86]:
sum(e)

1